In [4]:
from data_generator.main import get_real_data, generate_from_real_data
from data_generator.utils import plot_distribution_comparison
from methods.adf.main1 import adf_fairness_testing
from methods.utils import reformat_discrimination_results, convert_to_non_float_rows, compare_discriminatory_groups
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm


def get_groups(results_df_origin, data_obj, schema):
    non_float_df = convert_to_non_float_rows(results_df_origin, schema)
    predefined_groups_origin = reformat_discrimination_results(non_float_df, data_obj.dataframe)
    nb_elements = sum([el.group_size for el in predefined_groups_origin])
    return predefined_groups_origin, nb_elements

In [10]:
def run_experiment(dataset_name, original_params, synth_params):
    # Get original data
    data_obj, schema = get_real_data(dataset_name)

    # Run fairness testing on original data
    results_df_origin, metrics_origin = adf_fairness_testing(
        data_obj, **original_params
    )
    predefined_groups_origin, nb_elements = get_groups(results_df_origin, data_obj, schema)

    # Generate and test synthetic data
    data_obj_synth, schema = generate_from_real_data(dataset_name, nb_groups=100)
    results_df_synth, metrics_synth = adf_fairness_testing(
        data_obj_synth, **synth_params
    )
    predefined_groups_synth, nb_elements_synth = get_groups(results_df_synth, data_obj, schema)

    # Compare results
    comparison_results = compare_discriminatory_groups(predefined_groups_origin, predefined_groups_synth)

    return {
        'metrics_origin': metrics_origin,
        'metrics_synth': metrics_synth,
        'comparison_results': comparison_results,
        'nb_elements': nb_elements,
        'nb_elements_synth': nb_elements_synth
    }


def run_multiple_experiments(dataset_name, original_params, synth_params, num_runs=10):
    results_list = []

    for run in tqdm(range(num_runs), desc=f"Running experiments for {dataset_name}"):
        result = run_experiment(dataset_name, original_params, synth_params)
        metrics = {
            'dataset': dataset_name,
            'run': run,
            'coverage_ratio': result['comparison_results']['coverage_ratio'],
            'matched_groups': result['comparison_results']['total_groups_matched'],
            'total_groups': result['comparison_results']['total_original_groups'],
            'matched_size': result['comparison_results']['total_matched_size'],
            'total_size': result['comparison_results']['total_original_size']
        }
        results_list.append(metrics)

    return pd.DataFrame(results_list)


def plot_experiment_results(results_df):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Coverage ratio boxplot
    sns.boxplot(data=results_df, x='dataset', y='coverage_ratio', ax=axes[0, 0])
    axes[0, 0].set_title('Coverage Ratio Distribution')

    # Matched groups vs total groups
    for dataset in results_df['dataset'].unique():
        dataset_data = results_df[results_df['dataset'] == dataset]
        axes[0, 1].scatter(dataset_data['total_groups'], dataset_data['matched_groups'],
                           label=dataset, alpha=0.6)
    axes[0, 1].plot([0, results_df['total_groups'].max()], [0, results_df['total_groups'].max()],
                    'k--', alpha=0.3)
    axes[0, 1].set_title('Matched vs Total Groups')
    axes[0, 1].legend()

    # Size comparison
    results_df.groupby('dataset')[['matched_size', 'total_size']].mean().plot(
        kind='bar', ax=axes[1, 0])
    axes[1, 0].set_title('Average Matched vs Total Size')

    # Run variation
    for dataset in results_df['dataset'].unique():
        dataset_data = results_df[results_df['dataset'] == dataset]
        axes[1, 1].plot(dataset_data['run'], dataset_data['coverage_ratio'],
                        marker='o', label=dataset)
    axes[1, 1].set_title('Coverage Ratio by Run')
    axes[1, 1].legend()

    plt.tight_layout()
    return fig

In [ ]:
# Run experiments
datasets = ['bank']
all_results = []

original_params = {'max_global': 5000, 'max_local': 2000, 'max_iter': 10, 'cluster_num': 50, 'max_runtime_seconds': 300}
synth_params = {'max_global': 5000, 'max_local': 2000, 'max_iter': 10, 'cluster_num': 50, 'max_runtime_seconds': 300}

for dataset in datasets:
    dataset_results = run_multiple_experiments(dataset, original_params, synth_params, num_runs=1)
    all_results.append(dataset_results)

results_df = pd.concat(all_results)


Running experiments for bank:   0%|          | 0/1 [00:00<?, ?it/s]2025-02-28 17:48:34 - SingleTableSynthesizer - INFO - {'EVENT': 'Instance', 'TIMESTAMP': datetime.datetime(2025, 2, 28, 17, 48, 34, 482886), 'SYNTHESIZER CLASS NAME': 'GaussianCopulaSynthesizer', 'SYNTHESIZER ID': 'GaussianCopulaSynthesizer_1.18.0_f8b742f2ec314d0f991affe4d298ca74'}
2025-02-28 17:48:34 - SingleTableSynthesizer - INFO - {'EVENT': 'Fit', 'TIMESTAMP': datetime.datetime(2025, 2, 28, 17, 48, 34, 482886), 'SYNTHESIZER CLASS NAME': 'GaussianCopulaSynthesizer', 'SYNTHESIZER ID': 'GaussianCopulaSynthesizer_1.18.0_f8b742f2ec314d0f991affe4d298ca74', 'TOTAL NUMBER OF TABLES': 1, 'TOTAL NUMBER OF ROWS': 60000, 'TOTAL NUMBER OF COLUMNS': 17}
2025-02-28 17:48:34 - sdv.data_processing.data_processor - INFO - Fitting table  metadata
2025-02-28 17:48:34 - sdv.data_processing.data_processor - INFO - Fitting formatters for table 
2025-02-28 17:48:34 - sdv.data_processing.data_processor - INFO - Fitting constraints for table

Fitting GaussianCopulaSynthesizer...


2025-02-28 17:48:37 - SingleTableSynthesizer - INFO - {'EVENT': 'Fit processed data', 'TIMESTAMP': datetime.datetime(2025, 2, 28, 17, 48, 37, 242200), 'SYNTHESIZER CLASS NAME': 'GaussianCopulaSynthesizer', 'SYNTHESIZER ID': 'GaussianCopulaSynthesizer_1.18.0_f8b742f2ec314d0f991affe4d298ca74', 'TOTAL NUMBER OF TABLES': 1, 'TOTAL NUMBER OF ROWS': 60000, 'TOTAL NUMBER OF COLUMNS': 17}
2025-02-28 17:48:37 - copulas.multivariate.gaussian - INFO - Fitting GaussianMultivariate(distribution="{'Attr1_T': <class 'copulas.univariate.beta.BetaUnivariate'>, 'Attr2_X': <class 'copulas.univariate.truncated_gaussian.TruncatedGaussian'>, 'Attr3_T': <class 'copulas.univariate.beta.BetaUnivariate'>, 'Attr4_T': <class 'copulas.univariate.beta.BetaUnivariate'>, 'Attr5_X': <class 'copulas.univariate.truncated_gaussian.TruncatedGaussian'>, 'Attr6_X': <class 'copulas.univariate.truncated_gaussian.TruncatedGaussian'>, 'Attr7_X': <class 'copulas.univariate.truncated_gaussian.TruncatedGaussian'>, 'Attr8_X': <clas

Fitting GaussianCopulaSynthesizer...


2025-02-28 17:51:58 - SingleTableSynthesizer - INFO - {'EVENT': 'Fit processed data', 'TIMESTAMP': datetime.datetime(2025, 2, 28, 17, 51, 58, 230190), 'SYNTHESIZER CLASS NAME': 'GaussianCopulaSynthesizer', 'SYNTHESIZER ID': 'GaussianCopulaSynthesizer_1.18.0_d542c2b2b5ec46e0a84df465536ab108', 'TOTAL NUMBER OF TABLES': 1, 'TOTAL NUMBER OF ROWS': 45211, 'TOTAL NUMBER OF COLUMNS': 17}
2025-02-28 17:51:58 - copulas.multivariate.gaussian - INFO - Fitting GaussianMultivariate(distribution="{'Attr1_T': <class 'copulas.univariate.beta.BetaUnivariate'>, 'Attr2_X': <class 'copulas.univariate.truncated_gaussian.TruncatedGaussian'>, 'Attr3_T': <class 'copulas.univariate.beta.BetaUnivariate'>, 'Attr4_T': <class 'copulas.univariate.beta.BetaUnivariate'>, 'Attr5_X': <class 'copulas.univariate.truncated_gaussian.TruncatedGaussian'>, 'Attr6_X': <class 'copulas.univariate.truncated_gaussian.TruncatedGaussian'>, 'Attr7_X': <class 'copulas.univariate.truncated_gaussian.TruncatedGaussian'>, 'Attr8_X': <clas

Training GaussianCopulaSynthesizer on bank dataset...


2025-02-28 17:52:05 - SingleTableSynthesizer - INFO - {'EVENT': 'Fit processed data', 'TIMESTAMP': datetime.datetime(2025, 2, 28, 17, 52, 5, 755225), 'SYNTHESIZER CLASS NAME': 'GaussianCopulaSynthesizer', 'SYNTHESIZER ID': 'GaussianCopulaSynthesizer_1.18.0_ba409a19084d4d16a293f2b5b76a8d6d', 'TOTAL NUMBER OF TABLES': 1, 'TOTAL NUMBER OF ROWS': 45211, 'TOTAL NUMBER OF COLUMNS': 17}
2025-02-28 17:52:05 - copulas.multivariate.gaussian - INFO - Fitting GaussianMultivariate(distribution="{'Attr1_T': <class 'copulas.univariate.beta.BetaUnivariate'>, 'Attr2_X': <class 'copulas.univariate.truncated_gaussian.TruncatedGaussian'>, 'Attr3_T': <class 'copulas.univariate.beta.BetaUnivariate'>, 'Attr4_T': <class 'copulas.univariate.beta.BetaUnivariate'>, 'Attr5_X': <class 'copulas.univariate.truncated_gaussian.TruncatedGaussian'>, 'Attr6_X': <class 'copulas.univariate.truncated_gaussian.TruncatedGaussian'>, 'Attr7_X': <class 'copulas.univariate.truncated_gaussian.TruncatedGaussian'>, 'Attr8_X': <class

In [8]:
# Print summary statistics
print("\nSummary Statistics:")
print(results_df.groupby('dataset').agg({
    'coverage_ratio': ['mean', 'std'],
    'matched_groups': ['mean', 'std'],
    'total_groups': ['mean']
}).round(3))


Summary Statistics:
        coverage_ratio     matched_groups     total_groups
                  mean std           mean std         mean
dataset                                                   
adult              1.0 NaN           15.0 NaN         15.0
bank               0.0 NaN            0.0 NaN          4.0


In [ ]:
# Create visualizations
fig = plot_experiment_results(results_df)